In [ ]:
# Import lib
import cv2
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image
from tqdm import tqdm

In [ ]:
DATABASE = "/Users/Ombeline/Documents/IMT/M2/4 - UV Projet/python_reid/nimages"

# 1. Rotate images

Récupère les préfixes VID_001, VID_002

In [ ]:
def find_prefix_vid (global_fold):
  """
    This function searches for unique prefixes of folders in the global_fold directory.
  """
  prefixes_to_process = set()

  for fold in os.listdir(global_fold):
    dossier_path = os.path.join(global_fold, fold)
    if os.path.isdir(dossier_path):
        # Extract folder prefix
        prefix = fold.split("_", 1)[0] + "_" + fold.split("_", 1)[1].split("_")[0]
        prefixes_to_process.add(prefix)

  return prefixes_to_process

Récupère la liste des paths des dossiers qu'ilo faut rotate de 180 degrés

In [ ]:
def find_folds_to_rotate (global_fold):
  """
    This function finds the folders to process (to rotate) based on the prefixes found in the global_fold directory.
  """
  prefixes_to_process = find_prefix_vid(global_fold)
  folds_to_rotate = []

  # Browse VID_001, VID_002, etc
  for prefix in prefixes_to_process:

    # List of folders for each video (starting with VID_001, VID_002, etc)
    list_folds_prefix = os.path.join(global_fold, f"{prefix}*")
    folds = sorted(glob(list_folds_prefix))

    # Select the second half of the folders
    nbr_to_rotate = len(folds) // 2
    folds_to_rotate.extend(folds[nbr_to_rotate:])

  return folds_to_rotate

In [ ]:
folds_to_rotate = sorted(find_folds_to_rotate(DATABASE))

for fold_path in tqdm(folds_to_rotate, desc="Processing rotate images"):
    if os.path.exists(fold_path):
        for file in os.listdir(fold_path):
            file_path = os.path.join(fold_path, file)

            if os.path.isfile(file_path) and file.endswith(('.png')):
                image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)

                # Rotate image 180°
                rotated_image = np.rot90(image, k=2)
                rotated_image = Image.fromarray(rotated_image)
                rotated_image.save(file_path)

    else:
        print(f"Le dossier {fold_path} n'existe pas.")

# 2. Delete black band images

Retourne si l'image est refusée car trop bruitée (true) ou acceptée (false)

In [ ]:
def is_black_band_not_acceptable(depth_image, black_pixels_threshold):
  """
  Check if the black band in the depth image is acceptable.
  """

  black_pixels_count = np.sum(depth_image == 0)
  return black_pixels_count >= black_pixels_threshold

Retourne un dict avec les path des images à supprimer

In [ ]:
def filter_black_band_images_folder(global_fold, black_pixels_threshold):
  """
  Filter depth images in the folder based on the black band acceptability.
  """

  dict_to_delete = {}

  for fold in tqdm(os.listdir(global_fold), desc="Filter black band images"):
    subfold_path = os.path.join(global_fold, fold)
    delete_list = []
    for filename in os.listdir(subfold_path):
      if filename.endswith("_D.png"):
        image_path_depth = os.path.join(subfold_path, filename)
        depth_image = cv2.imread(image_path_depth, cv2.IMREAD_UNCHANGED)

        if is_black_band_not_acceptable(depth_image, black_pixels_threshold):
          image_path_rgb = image_path_depth.replace('_D.png', '_RGB.png')
          delete_list.append(image_path_depth)
          delete_list.append(image_path_rgb)

    dict_to_delete[subfold_path] = sorted(delete_list)

  return dict_to_delete

In [ ]:
def delete_from_dict(dict_of_path):
  for subfold, paths in dict_of_path.items():
    for path in paths:
      os.remove(path)

In [ ]:
threshold_black_band = int(150 * 150 * 0.2)
image_dict_to_delete = filter_black_band_images_folder(DATABASE, threshold_black_band)
delete_from_dict(image_dict_to_delete)

# 3. Delete no human images

Vérifie si il y a un humain = si il y a une profondeur < 2000 (une profondeur entre la caméra et la table)

In [ ]:
def is_human(depth_image, height_threshold):
  """
  Check if there is a human
  """
  depth_non_zero = depth_image[depth_image != 0]

  lower_value = np.unique(np.sort(depth_non_zero))[0]

  return lower_value <= height_threshold

Retourne un dict avec les paths des imaegs à supprimer lorsqu'il n'y a pas d'humain

In [ ]:
def filter_images_no_human(global_fold, height_threshold):
  """
  Filter depth images in the folder based on the human acceptability.
  """

  dict_to_delete = {}

  for fold in tqdm(os.listdir(global_fold), desc="Filter human images"):
    subfold_path = os.path.join(global_fold, fold)
    delete_list = []
    for filename in os.listdir(subfold_path):
      if filename.endswith("_D.png"):
        image_path_depth = os.path.join(subfold_path, filename)
        depth_image = cv2.imread(image_path_depth, cv2.IMREAD_UNCHANGED)

        if not is_human(depth_image, height_threshold):
          image_path_rgb = image_path_depth.replace('_D.png', '_RGB.png')
          delete_list.append(image_path_depth)
          delete_list.append(image_path_rgb)

    dict_to_delete[subfold_path] = sorted(delete_list)

  return dict_to_delete

In [ ]:
height_threshold = 2000
image_dict_to_delete = filter_images_no_human(DATABASE, height_threshold)
delete_from_dict(image_dict_to_delete)